In [41]:
# Model of the form
# min    c^T . x + E[min q(omega)^T . y(omega)]
# s.t.   A.x <= b
#        x >= 0,
#        T(omega).x + W(omega)y(omega)<=h(omega)     for all omega
#        y(omega)>= 0

# Deterministic
A = [1, 1, 1]
b = 500
c = [150,230,260]

# Scenario Probability
p = ones(3) * 1.0/3.0
q = [238, 210, -170, -150, -36, -10]

# T matrix (coefficient of x in scenarios)
prod=[[3.0, 3.6, 24],
      [2.5, 3.0, 20],
      [2.0, 2.4, 16]]
T = zeros(3,4,3)  # #scenarios X #constraint #variables
for scenario in 1:3
    for crops in 1:3
        T[scenario,crops,crops] = -prod[scenario][crops]
    end
end

# W matrix (coefficient of y in scenarios)
W = zeros(4,6)
for i in 1:2
    W[i  ,i  ] = - 1
    W[i,i+2] = 1
end
W[3,5] = 1
W[3,6] = 1
W[4,5] = 1

# h (right-hand term in scenarios)
h = [-200,-240,0,6000]

4-element Vector{Int64}:
 -200
 -240
    0
 6000

In [46]:
using JuMP, Gurobi
solver = optimizer_with_attributes(() -> Gurobi.Optimizer(),"InfUnbdInfo" => 1, "OutputFlag" => 0, "Method" => 0)


v_o_k = zeros(3,4,1000)  # Vertexes (scenario, constraint, number)
r_o_k = zeros(3,4,1000)  # Extreme rays (scenario, constraint, number)
n_v_o_k = 0              # Number of added vertexes
n_r_o_k = 0              # Number of added extreme rays


function create_master(model::Model, lower_bound::Int64)
    @variable(model, theta)
    @constraint(model, theta >= lower_bound)
    @variable(model, x[1:3] >= 0)
    @constraint(model, c_master, sum(A[n] * x[n] for n in 1:3) <= b)
    @objective(model, Min, sum(c[n] * x[n] for n in 1:3) + theta)
end


print("\nMaster Model Creation")
master_model = Model(solver)

# empty model is initialized
create_master(master_model,0)

print("\nMaster Model Optimization")
optimize!(master_model)
xi = value.(master_model[:x])
print(xi)
#function add_feasibility_cut(model::Model, scenario::Int64, r_k)
#    @constraint(model, sum(r_k[i] * (h[i] * sum(T[scenario, i,j] * model[:x][j]) for j in eachindex(x) for i in 1:4)) <= 0)
#    r_o_k[scenario][i] = r_k[i] for i in 1:4
    
#end


#function add_optimality_cut(model::Model, v_k)
#    @constraint(model, sum(p[k] * sum(v_k[k,i] * (h[i] - sum(T[k, i,j] * model[:x][j] for j in eachindex(x)) ) for i in 1:4) for k in 1:3) <= model[:theta])
#    v_o_k[k,i] = v_k[k,i] for i in 1:4 for k in 1:3
#end



# Let's create slaves

function create_slave(model::Model, scenario)
    @variable(model, y[1:6] >= 0)
    @variable(model, x[1:3] >= 0)
    @constraint(model, constr[n in 1:4], sum(W[n,i] * y[i] for i in eachindex(y)) <= h[n] - sum(T[scenario,n,i] * x[i] for i in eachindex(x)))
    @objective(model, Min, sum(q[i] * y[i] for i in eachindex(y)))
end


slave_models = Array{Model}(undef, 3)

for i in 1:3
    print("\n\nSlave Model Creation : ")
    print(i)
    print("\n")
    
    slave_models[i] = Model(solver)
    create_slave(slave_models[i], i)
    
    for j in 1:3
        fix(slave_models[i][:x][j], xi[j]; force=true)
    end
    optimize!(slave_models[i])
    
    
    
    if termination_status(slave_models[i]) == DUAL_INFEASIBLE && primal_status(slave_models[i]) == INFEASIBILITY_CERTIFICATE
        print("\nDual Infeasible")
        # Retrieving an extreme ray if the primal problem is unbounded
        an_extreme_ray = value.(slave_models[i][:x])
        print(an_extreme_ray)
        # Check the values returned. Could Other extreme rays  have been retuned ?
    elseif termination_status(slave_models[i]) == OPTIMAL
        print("\nOptimal\n")
        # Retrieving an optimal solution
        an_optimal_point = value.(slave_models[i][:x])
        print(an_optimal_point)
        print("\n")
        an_optimal_point = value.(slave_models[i][:y])
        print(an_optimal_point)
end
    
end

# Note that this initializations has to be run just 1 time. The models are now stored and we can
# Iterate between master and slave

optimize!(master_model)
 


Master Model CreationSet parameter Username
Academic license - for non-commercial use only - expires 2024-03-30
Set parameter InfUnbdInfo to value 1

Master Model OptimizationSet parameter Method to value 0
Set parameter InfUnbdInfo to value 1
[0.0, 0.0, 0.0]

Slave Model Creation : 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-30
Set parameter InfUnbdInfo to value 1
Set parameter Method to value 0
Set parameter InfUnbdInfo to value 1

Optimal
[0.0, 0.0, 0.0]
[200.0, 240.0, 0.0, 0.0, 0.0, 0.0]

Slave Model Creation : 2
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-30
Set parameter InfUnbdInfo to value 1
Set parameter Method to value 0
Set parameter InfUnbdInfo to value 1

Optimal
[0.0, 0.0, 0.0]
[200.0, 240.0, 0.0, 0.0, 0.0, 0.0]

Slave Model Creation : 3
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-30
Set parameter InfUnbdInfo to value 1
Set parameter Metho

In [39]:
# Load optimization packages
using JuMP, Gurobi

# Let's define solver with attributes
solver = optimizer_with_attributes(() -> Gurobi.Optimizer(), "OutputFlag" => 0, "Method" => 0)

function create_master(model::Model)
    @variable(model, x[1:3] >= 0)
    @variable(model, y[1:2] >= 0)

    @constraint(model, constr[n in 1:2], x[n]+y[n] >= n)
    @objective(model, Min, sum(x[n] for n in 1:2))
end

# empty model
master_model = Model(solver)

# empty model is initialized
create_master(master_model)

function initialize_master(model::Model, lower_bound::Int64)
    @variable(model, theta)
    @constraint(model, theta >= lower_bound)
    objective = objective_function(model)
    sense = objective_sense(model)
    @objective(model, sense, objective + theta)
end

initialize_master(master_model, 0)
optimize!(master_model)

# Let's create slaves

function create_slave(model::Model, xi)
    @variable(model, z[1:2] >= 0)
    @variable(model, x[1:3] )
    @variable(model, y[1:2] )

    @constraint(model, constr[n in 1:2], z[n] >= xi[n] + x[n])
    @objective(model, Min, sum(z[n] for n in 1:2))
end

xi = [[2,3], [4,5]]

slave_models = Array{Model}(undef, length(xi))

for i in 1:length(xi)
    curr_xi = xi[i]
    slave_models[i] = Model(solver)
    create_slave(slave_models[i], curr_xi)
end

# Note that this initializations has to be run just 1 time. The models are now stored and we can
# Iterate between master and slave

optimize!(master_model)
current_x = value.(master_model[:x])
current_y = value.(master_model[:y])

for i in 1:length(xi)

    # we fix the x variable to the current value
    x = slave_models[i][:x]
    for j in eachindex(x)
        fix(x[j], current_x[j]; force=true)
    end

    # we fix the y variable to the current value
    y = slave_models[i][:y]
    for j in eachindex(y)
        fix(y[j], current_y[j]; force=true)
    end

    optimize!(slave_models[i])

    # extract dual and build cuts
end


Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-30
Set parameter Method to value 0
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-30
Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-30
Set parameter Method to value 0
Set parameter Method to value 0


In [31]:
using JuMP, Gurobi

solver = optimizer_with_attributes(() -> Gurobi.Optimizer(),"InfUnbdInfo" => 1,"OutputFlag" => 0, "Method" => 0)
m = Model(solver)
@variable(m, x[1:2])
@constraint(m, c[i=1:2], x[i] >= 2)
# Uncommenting the next line leads to an optimal solution instead of an unbounded problem...
@constraint(m, c2[i=2:2], x[i] <= 5)
@objective(m, Max, sum(x))
optimize!(m)

if termination_status(m) == DUAL_INFEASIBLE && primal_status(m) == INFEASIBILITY_CERTIFICATE
    print("\nDual Infeasible")
    # Retrieving an extreme ray if the primal problem is unbounded
    an_extreme_ray = value.(x)
    # Check the values returned. Could Other extreme rays  have been retuned ?
elseif termination_status(m) == OPTIMAL
    print("Optimal")
    # Retrieving an optimal solution
    an_optimal_point = value.(x)
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-30
Set parameter InfUnbdInfo to value 1
Set parameter Method to value 0
Set parameter InfUnbdInfo to value 1

Dual Infeasible

2-element Vector{Float64}:
 1.0
 0.0